# Exploring Random Forests

In [21]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.datasets import load_boston, load_iris, load_wine, load_digits, \
                             load_breast_cancer, load_diabetes

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'

from rfpimp import *

from sklearn import tree
from dtreeviz.trees import *

In [36]:
def rent(n=None, bootstrap=False):
    df_rent = pd.read_csv("data/rent-ideal.csv")
    if n is None:
        n = len(df_rent)
    df_rent = df_rent.sample(n, replace=bootstrap)
    X = df_rent[['bedrooms','bathrooms','latitude','longitude']]
    y = df_rent['price']
    return X, y

def boston():
    boston = load_boston()
    X = boston.data
    y = boston.target
    features = boston.feature_names
    df = pd.DataFrame(data=X,columns=features)
    df['y'] = y
    return df

## Set up

Get the `rent-ideal.csv`  data file from canvas and store in the data directory underneath your notebook  directory.

In [100]:
X, y = rent()
X.head(3)

,bedrooms,bathrooms,latitude,longitude
35374,2,2.0,40.7176,-73.9531
2603,3,1.0,40.7073,-74.0046
34591,1,1.0,40.7283,-73.9857


In [162]:
X.shape

(48300, 4)

## Train random forests of different sizes

In [101]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

Here's how to train a random forest  that has a single tree:

In [136]:
rf = RandomForestRegressor(n_estimators=1)
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=1, n_jobs=None,
                      oob_score=False, random_state=None, verbose=0,
                      warm_start=False)

Compute the MAE for the training and the testing set, printing them out.

In [137]:
mae_train = mean_absolute_error(y_train, rf.predict(X_train))
mae = mean_absolute_error(y_test, rf.predict(X_test))
print(f"MAE train {mae_train:.1f}$, test {mae:.1f}$")

MAE train 202.7$, test 339.2$


<details>
<summary>Solution</summary>
<pre>
mae_train = mean_absolute_error(y_train, rf.predict(X_train))
mae = mean_absolute_error(y_test, rf.predict(X_test))
</pre>
</details>

Run the training and testing cycle several times to see the variance: the test scores bounce around a lot.

Increase the number of trees (`n_estimators`) to 2, retrain, and print out the results.

In [146]:
rf = RandomForestRegressor(n_estimators=2)
rf.fit(X_train, y_train)
mae_train = mean_absolute_error(y_train, rf.predict(X_train))
mae = mean_absolute_error(y_test, rf.predict(X_test))
print(f"MAE train {mae_train:.1f}$, test {mae:.1f}$")

MAE train 199.3$, test 323.0$


You should notice the both test MAE scores going down and bouncing around less from run to run.

**Q.**  Why does the MAE score go down?

<details>
<summary>Solution</summary>
    With 2 trees, the chances are that the random forest will have seen (trained on) more of the original training set, despite bootstrapping.
</details>

Increase the number of trees (`n_estimators`) to 10, retrain, and print out the results.

In [153]:
rf = RandomForestRegressor(n_estimators=10)
rf.fit(X_train, y_train)
mae_train = mean_absolute_error(y_train, rf.predict(X_train))
mae = mean_absolute_error(y_test, rf.predict(X_test))
print(f"MAE train {mae_train:.1f}$, test {mae:.1f}$")

MAE train 189.3$, test 299.3$


**Q.**  What you notice about the MAE scores?

<details>
<summary>Solution</summary>
They are getting smaller.
</details>

**Q.**  After running several times, what else do you notice?

<details>
<summary>Solution</summary>
    With 10 trees, the prediction from run to run varies a lot less. We have reduced variance,  improving generality.
</details>

Increase the number of trees (`n_estimators`) to 100, retrain, and print out the results.

In [160]:
rf = RandomForestRegressor(n_estimators=200)
%time rf.fit(X_train, y_train) # how long does this take?
mae_train = mean_absolute_error(y_train, rf.predict(X_train))
mae = mean_absolute_error(y_test, rf.predict(X_test))
print(f"MAE train {mae_train:.1f}$, test {mae:.1f}$")

CPU times: user 8.39 s, sys: 157 ms, total: 8.55 s
Wall time: 8.56 s
MAE train 184.5$, test 292.0$


**Q.**  What you notice about the MAE scores from a single run?

<details>
<summary>Solution</summary>
They are a bit smaller, but not by much.
</details>

Notice that it took a long time to train, about 10 seconds.  Do the exact same thing again but this time use `n_jobs=-1` as an argument to the `RandomForestRegressor` constructor. This tells the library to use all processing cores available on the computer processor. As long as the data is not too huge (because it must pass it around), it often goes much faster using this argument. It should take less than two seconds.

In [159]:
rf = RandomForestRegressor(n_estimators=200, n_jobs=-1)
%time rf.fit(X_train, y_train)
mae_train = mean_absolute_error(y_train, rf.predict(X_train))
mae = mean_absolute_error(y_test, rf.predict(X_test))
print(f"MAE train {mae_train:.1f}$, test {mae:.1f}$")

CPU times: user 13.1 s, sys: 302 ms, total: 13.4 s
Wall time: 1.86 s
MAE train 184.4$, test 292.3$


**Q.**  What you notice about the MAE scores from SEVERAL runs?

<details>
<summary>Solution</summary>
The variance is even lower (tighter).
</details>

## Altering bootstrap size

Jeremy Howard has an awesome trick that he uses during development of a model to convince sklearn random forests to use less than $n$ observations from the training data to train each tree.

In [177]:
def jeremy_trick_RF_sample_size(n):
    from sklearn.ensemble import forest
    forest._generate_sample_indices = \
        (lambda rs, n_samples: forest.check_random_state(rs).randint(0, n_samples,n))

There are about 38,000 training records, change that to 19,000 and check the accuracy again.

In [178]:
jeremy_trick_RF_sample_size(round(len(X_train)/2))

In [182]:
rf = RandomForestRegressor(n_estimators=200, n_jobs=-1)
%time rf.fit(X_train, y_train)
mae_train = mean_absolute_error(y_train, rf.predict(X_train))
mae = mean_absolute_error(y_test, rf.predict(X_test))
print(f"MAE train {mae_train:.1f}$, test {mae:.1f}$")

CPU times: user 8.49 s, sys: 187 ms, total: 8.67 s
Wall time: 1.27 s
MAE train 227.3$, test 308.3$


It's a less accurate, but it's faster. (Down to 1.27s from 1.86s on my machine.)

**Q.**  Why is it less accurate?

<details>
<summary>Solution</summary>
Each tree is seeing less of the data set during training.
</details>

Turn off bootstrapping by adding `bootstrap=False` to the constructor of the model. This means that it will subsample rather than bootstrap. Remember that bootstrapping gets about two thirds of the data because of replacement.

In [183]:
rf = RandomForestRegressor(n_estimators=200, n_jobs=-1, bootstrap=False)
%time rf.fit(X_train, y_train)
mae_train = mean_absolute_error(y_train, rf.predict(X_train))
mae = mean_absolute_error(y_test, rf.predict(X_test))
print(f"MAE train {mae_train:.1f}$, test {mae:.1f}$")

CPU times: user 17.9 s, sys: 365 ms, total: 18.2 s
Wall time: 2.59 s
MAE train 137.2$, test 301.8$


That brings the accuracy backup a little bit for the test set but very much so for the training MAE score.

Drop that size to one third of the training records then retrain and test.

In [184]:
jeremy_trick_RF_sample_size(round(len(X_train)/3))

In [193]:
rf = RandomForestRegressor(n_estimators=200, n_jobs=-1)
%time rf.fit(X_train, y_train)
mae_train = mean_absolute_error(y_train, rf.predict(X_train))
mae = mean_absolute_error(y_test, rf.predict(X_test))
print(f"MAE train {mae_train:.1f}$, test {mae:.1f}$")

CPU times: user 6.33 s, sys: 163 ms, total: 6.5 s
Wall time: 965 ms
MAE train 256.2$, test 321.5$


Mine is twice as fast as the full bootstrap but continues to have very tight variance because of the number of trees. The accuracy is lower, however, about what we get for the  usual random forest with two trees.

Before we forget, let's reset the bootstrapping size back to the usual.

In [200]:
def jeremy_trick_reset_RF_sample_size():
    forest._generate_sample_indices = (lambda rs, n_samples:
        forest.check_random_state(rs).randint(0, n_samples, n_samples))

In [201]:
jeremy_trick_reset_RF_sample_size()

## RF prediction confidence

A random forest is a collection of decision trees, each of which contributes a prediction. The forest averages those predictions to provide the overall prediction. Let's dig inside the random forest to get the individual trees out and ask them what their predictions are.

Train a random forest with 10 trees on `X_train`, `y_train`.  Use `for t in rf.estimators_` to iterate through the trees making predictions with `t` not `rf`. Print out the usual MAE scores for each tree predictor.

In [203]:
rf = RandomForestRegressor(n_estimators=10, n_jobs=-1)
rf.fit(X_train, y_train)

for t in rf.estimators_:
    mae_train = mean_absolute_error(y_train, t.predict(X_train))
    mae = mean_absolute_error(y_test, t.predict(X_test))
    print(f"MAE train {mae_train:.1f}$, test {mae:.1f}$")
#print(t.score(y_test, t.predict(X_test)))

MAE train 205.4$, test 343.5$
MAE train 204.4$, test 339.3$
MAE train 203.9$, test 339.3$
MAE train 202.2$, test 340.4$
MAE train 204.0$, test 345.1$
MAE train 207.8$, test 347.2$
MAE train 205.7$, test 337.6$
MAE train 205.0$, test 348.5$
MAE train 204.9$, test 338.1$
MAE train 208.2$, test 341.3$


Notice that it bounces around quite a bit. 

Select one of the `X_test` rows and print out the addicted rent price.

In [249]:
x = X_test.iloc[3,:] # pick single test case
x = x.values.reshape(1,-1)

print(f"{x} => {rf.predict(x)}$")

[[  0.       1.      40.7248 -73.9922]] => [4256.74166667]$


Now let's see how the forest came to that conclusion. Compute the average of the predictions obtained from every tree.  Compare that to the prediction obtained directly from the random forest (`rf.predict(X_test)`). They should be the same.

In [250]:
y_pred = np.mean([t.predict(x) for t in rf.estimators_])
print(f"{x} => {y_pred}$")

[[  0.       1.      40.7248 -73.9922]] => 4256.741666666667$


Compute the standard deviation of the tree estimates and print that out.

In [237]:
np.std([t.predict(x) for t in rf.estimators_])

235.94656883728663

The lower the standard deviation, the more tightly grouped the predictions were, which means we should have more confidence in our answer. 

Different records will often have different standard deviations, which means we could have different levels of confidence in the various answers. This might be helpful to a bank for example that wanted to not only predict whether to give loans, but how confident the model was.